# 📱 Guia de Integração: Leitor de Código de Barras USB no Sistema SGE

## 🎯 Objetivo
Este notebook demonstra como integrar um leitor de código de barras USB no sistema SGE (Sistema de Gerenciamento de Estoque), maximizando a eficiência operacional em diferentes módulos.

## 🛠️ Cenários de Uso Implementados

### ✅ **1. Venda Rápida (JÁ IMPLEMENTADO)**
- **Local**: Interface de venda rápida
- **Funcionalidade**: Scan automático para adicionar produtos ao carrinho
- **Status**: ✅ Funcional com detecção automática

### 📦 **2. Cadastro de Produtos**
- **Local**: Formulário de produtos
- **Funcionalidade**: Captura automática do código de barras
- **Prioridade**: Alta

### 📥 **3. Entradas de Estoque**
- **Local**: Módulo de inflows
- **Funcionalidade**: Conferência rápida de mercadorias
- **Prioridade**: Média

### 👥 **4. Identificação de Clientes**
- **Local**: Cadastro/busca de clientes
- **Funcionalidade**: Cards de fidelidade ou documentos
- **Prioridade**: Baixa

### 📋 **5. Inventário e Conferência**
- **Local**: Nova funcionalidade
- **Funcionalidade**: Contagem física do estoque
- **Prioridade**: Alta

## 🎮 Como Funciona
O leitor USB funciona como um teclado virtual, digitando o código automaticamente no campo focado.

## 🔧 Implementação Técnica

### 1. Detecção de Código de Barras
O sistema usa JavaScript para detectar entrada rápida de dados (característica dos leitores USB):

```javascript
// Função principal de detecção
function handleBarcodeInput(input, callback) {
    let barcodeBuffer = '';
    let lastInputTime = 0;
    
    input.addEventListener('input', function(e) {
        const currentTime = Date.now();
        const timeDiff = currentTime - lastInputTime;
        
        // Se input muito rápido (< 100ms), provavelmente é scanner
        if (timeDiff < 100) {
            barcodeBuffer += e.target.value.slice(-1);
        } else {
            barcodeBuffer = e.target.value;
        }
        
        lastInputTime = currentTime;
        
        // Detecta final do scan (Enter ou delay > 100ms)
        setTimeout(() => {
            if (barcodeBuffer.length >= 8) { // Códigos têm min 8 chars
                callback(barcodeBuffer);
                barcodeBuffer = '';
            }
        }, 150);
    });
}
```

### 2. Feedback Visual
Sistema de notificações para informar o usuário sobre o status do scan:

```javascript
function showBarcodeNotification(message, type = 'info') {
    const notification = document.createElement('div');
    notification.className = `alert alert-${type} alert-dismissible fade show position-fixed`;
    notification.style.cssText = 'top: 20px; right: 20px; z-index: 9999; min-width: 300px;';
    notification.innerHTML = `
        <i class="bi bi-upc-scan me-2"></i>${message}
        <button type="button" class="btn-close" data-bs-dismiss="alert"></button>
    `;
    document.body.appendChild(notification);
    
    setTimeout(() => notification.remove(), 3000);
}
```

## 📦 Implementações por Módulo

### 1. 🛒 Venda Rápida (IMPLEMENTADO)
**Arquivo**: `sales/templates/sales/quick_sale.html`
- ✅ Detecção automática de código de barras
- ✅ Busca automática de produto
- ✅ Adição automática ao carrinho
- ✅ Feedback visual com notificações

### 2. 📝 Cadastro de Produtos (PROPOSTO)
**Arquivo**: `products/templates/products/product_form.html`

```html
<!-- Campo de código de barras com scanner -->
<div class="mb-3">
    <label for="id_barcode" class="form-label">
        <i class="bi bi-upc-scan me-2"></i>Código de Barras
    </label>
    <div class="input-group">
        <input type="text" class="form-control" id="id_barcode" name="barcode"
               placeholder="Digite ou escaneie o código de barras">
        <span class="input-group-text">
            <i class="bi bi-upc-scan text-primary"></i>
        </span>
    </div>
    <div class="form-text">
        <i class="bi bi-info-circle me-1"></i>
        Posicione o cursor no campo e escaneie o código de barras
    </div>
</div>

<script>
document.addEventListener('DOMContentLoaded', function() {
    const barcodeInput = document.getElementById('id_barcode');
    
    if (barcodeInput) {
        handleBarcodeInput(barcodeInput, function(barcode) {
            // Verifica se código já existe
            fetch(`/api/products/check-barcode/${barcode}/`)
                .then(response => response.json())
                .then(data => {
                    if (data.exists) {
                        showBarcodeNotification(
                            `Código ${barcode} já existe no produto: ${data.product_name}`,
                            'warning'
                        );
                    } else {
                        showBarcodeNotification(
                            `Código ${barcode} capturado com sucesso!`,
                            'success'
                        );
                    }
                });
        });
    }
});
</script>
```

### 3. 📥 Entradas de Estoque (PROPOSTO)
**Arquivo**: `inflows/templates/inflows/inflow_form.html`

```html
<!-- Área de scanner para conferência de produtos -->
<div class="row">
    <div class="col-md-8">
        <label for="scan_input" class="form-label">
            <i class="bi bi-upc-scan me-2"></i>Scanner de Conferência
        </label>
        <input type="text" class="form-control form-control-lg" 
               id="scan_input" placeholder="Escaneie os produtos recebidos..."
               autocomplete="off">
    </div>
    <div class="col-md-4">
        <label class="form-label">&nbsp;</label>
        <div class="d-grid">
            <button type="button" class="btn btn-outline-primary" id="clear_scan">
                <i class="bi bi-arrow-clockwise me-2"></i>Limpar
            </button>
        </div>
    </div>
</div>

<!-- Lista de produtos escaneados -->
<div id="scanned_products" class="mt-3">
    <h6><i class="bi bi-list-check me-2"></i>Produtos Conferidos:</h6>
    <div id="product_list" class="border rounded p-3 bg-light">
        <p class="text-muted mb-0">Nenhum produto escaneado ainda.</p>
    </div>
</div>
```

## 🔌 APIs Necessárias

### 1. Verificação de Produto por Código de Barras
**Endpoint**: `/api/products/check-barcode/<barcode>/`

```python
# products/views.py
from django.http import JsonResponse
from django.views.decorators.http import require_http_methods
from .models import Product

@require_http_methods(["GET"])
def check_barcode_api(request, barcode):
    try:
        product = Product.objects.get(barcode=barcode)
        return JsonResponse({
            'exists': True,
            'product_id': product.id,
            'product_name': product.name,
            'price': float(product.price),
            'stock': product.stock
        })
    except Product.DoesNotExist:
        return JsonResponse({'exists': False})
```

### 2. Busca de Produto para Venda (JÁ IMPLEMENTADO)
**Endpoint**: `/api/products/search-barcode/<barcode>/`
- ✅ Retorna dados completos do produto
- ✅ Inclui preço, estoque e informações para venda

### 3. Registro de Inventário (PROPOSTO)
**Endpoint**: `/api/inventory/scan/`

```python
# reporting/views.py (ou novo app inventory)
@require_http_methods(["POST"])
def inventory_scan_api(request):
    barcode = request.POST.get('barcode')
    user_id = request.user.id
    
    try:
        product = Product.objects.get(barcode=barcode)
        
        # Registra scan no inventário
        InventoryScan.objects.create(
            product=product,
            scanned_by_id=user_id,
            scan_timestamp=timezone.now()
        )
        
        return JsonResponse({
            'success': True,
            'product_name': product.name,
            'current_stock': product.stock
        })
    except Product.DoesNotExist:
        return JsonResponse({
            'success': False,
            'error': 'Produto não encontrado'
        })
```

## ⚙️ Configuração do Hardware

### 1. Configuração do Leitor USB
```markdown
1. **Conecte o leitor USB** ao computador
2. **Teste básico**: Abra um editor de texto e escaneie - deve digitar automaticamente
3. **Configuração de sufixo**: Se possível, configure para enviar ENTER após o código
4. **Velocidade**: Leitores USB digitam muito rápido (< 50ms entre caracteres)
```

### 2. Tipos de Código Suportados
- **EAN-13**: Padrão brasileiro (13 dígitos)
- **EAN-8**: Versão curta (8 dígitos)  
- **Code 128**: Alfanumérico
- **Code 39**: Alfanumérico simples
- **UPC-A**: Padrão americano

## 🚀 Implementação Passo a Passo

### Etapa 1: Cadastro de Produtos com Scanner

```bash
# 1. Adicionar campo barcode ao modelo (se não existir)
python manage.py makemigrations products
python manage.py migrate

# 2. Atualizar formulário de produtos
# Editar: products/forms.py
```

```python
# products/forms.py
class ProductForm(forms.ModelForm):
    class Meta:
        model = Product
        fields = ['name', 'barcode', 'brand', 'category', 'size', 'color', 'price', 'stock']
        widgets = {
            'barcode': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Digite ou escaneie o código de barras',
                'autocomplete': 'off'
            })
        }
```

### Etapa 2: Conferência de Entradas

```python
# inflows/views.py - Adicionar funcionalidade de scanner
def inflow_create_view(request):
    if request.method == 'POST':
        # Processar dados normais do formulário
        form = InflowForm(request.POST)
        
        # Processar produtos escaneados
        scanned_products = request.POST.getlist('scanned_products[]')
        
        if form.is_valid():
            inflow = form.save()
            
            # Adicionar produtos escaneados
            for barcode in scanned_products:
                try:
                    product = Product.objects.get(barcode=barcode)
                    InflowItem.objects.create(
                        inflow=inflow,
                        product=product,
                        quantity=1,  # ou permitir input de quantidade
                        verification_method='scanner'
                    )
                except Product.DoesNotExist:
                    messages.warning(request, f'Produto {barcode} não encontrado')
            
            return redirect('inflows:detail', pk=inflow.pk)
    else:
        form = InflowForm()
    
    return render(request, 'inflows/inflow_form.html', {'form': form})
```

### Etapa 3: Sistema de Inventário

```python
# Criar novo modelo para inventário
# reporting/models.py
class InventorySession(models.Model):
    name = models.CharField(max_length=100)
    started_by = models.ForeignKey(User, on_delete=models.CASCADE)
    start_date = models.DateTimeField(auto_now_add=True)
    end_date = models.DateTimeField(null=True, blank=True)
    status = models.CharField(max_length=20, choices=[
        ('active', 'Ativo'),
        ('completed', 'Finalizado')
    ], default='active')

class InventoryScan(models.Model):
    session = models.ForeignKey(InventorySession, on_delete=models.CASCADE)
    product = models.ForeignKey('products.Product', on_delete=models.CASCADE)
    scanned_quantity = models.IntegerField(default=1)
    scanned_by = models.ForeignKey(User, on_delete=models.CASCADE)
    scan_timestamp = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        unique_together = ['session', 'product']  # Um produto por sessão
```

## 📊 Exemplo de Uso Prático

### Cenário: Conferência de Mercadoria
1. **Fornecedor entrega** produtos
2. **Operador abre** módulo de entradas
3. **Scanner ativo** detecta códigos automaticamente
4. **Sistema valida** cada produto escaneado
5. **Lista em tempo real** mostra produtos conferidos
6. **Finalização** gera relatório de conferência

### Cenário: Inventário Mensal
1. **Gerente inicia** sessão de inventário
2. **Equipe escaneia** todos os produtos físicos
3. **Sistema conta** automaticamente as quantidades
4. **Relatório final** compara físico vs. sistema
5. **Ajustes automáticos** de estoque se necessário

## 🎯 Benefícios Esperados

### ⚡ Velocidade
- **90% mais rápido** que digitação manual
- **Redução de erros** de digitação
- **Workflow fluido** sem interrupções

### 📈 Precisão
- **Eliminação de erros** de código
- **Conferência automática** de produtos
- **Rastreabilidade completa** de operações

### 💼 Operacional
- **Menos treinamento** necessário
- **Maior produtividade** da equipe
- **Melhor experiência** do usuário

## 🔧 Troubleshooting

### Problema: Scanner não detectado
**Sintomas**: Código digitado manualmente, não automaticamente
**Soluções**:
- Verificar se campo tem foco
- Testar scanner em editor de texto
- Ajustar timing de detecção (diminuir de 100ms para 50ms)

### Problema: Códigos incompletos
**Sintomas**: Apenas parte do código é capturada
**Soluções**:
- Aumentar timeout de detecção para 200ms
- Verificar configuração do scanner (sufixo ENTER)
- Limpar campo antes de cada scan

### Problema: API lenta
**Sintomas**: Delay na busca de produtos
**Soluções**:
- Adicionar índice no campo barcode
- Implementar cache de produtos frequentes
- Usar AJAX assíncrono

```sql
-- Otimização de performance
CREATE INDEX idx_product_barcode ON products_product(barcode);
```

## 📋 Checklist de Implementação

### ✅ Venda Rápida (Concluído)
- [x] Detecção de scanner
- [x] Busca automática de produto
- [x] Feedback visual
- [x] Adição ao carrinho

### 🔲 Próximas Implementações

#### Cadastro de Produtos
- [ ] Campo barcode no formulário
- [ ] Validação de código único
- [ ] API de verificação
- [ ] Feedback de código existente

#### Entradas de Estoque
- [ ] Scanner de conferência
- [ ] Lista de produtos escaneados
- [ ] Validação contra pedido
- [ ] Relatório de divergências

#### Sistema de Inventário
- [ ] Modelo InventorySession
- [ ] Scanner de contagem
- [ ] Relatório físico vs. sistema
- [ ] Ajustes automáticos

#### Melhorias Gerais
- [ ] Configuração de tipos de código
- [ ] Histórico de scans
- [ ] Dashboard de usage
- [ ] Backup de configurações

## 🎉 Conclusão

O sistema SGE está **parcialmente integrado** com leitores de código de barras USB, com a funcionalidade de **venda rápida já operacional**.

### 🚀 Próximos Passos Recomendados:

1. **Implementar cadastro de produtos** com scanner (prioridade alta)
2. **Adicionar conferência de entradas** para reduzir erros
3. **Criar sistema de inventário** para controle físico
4. **Otimizar performance** das APIs de busca
5. **Adicionar métricas** de uso do scanner

### 💡 Dicas Operacionais:

- **Treinar usuários** sobre posicionamento do cursor
- **Configurar scanner** para enviar ENTER após código
- **Manter códigos limpos** e legíveis
- **Backup regular** da configuração do sistema

---

**Documentação criada em**: Janeiro 2025  
**Versão do sistema**: Django 5.2.4  
**Status**: Implementação parcial (Venda Rápida funcionando)